# Quickstart to integrate Recommenders in AzureML Designer

This notebook shows how to integrate any algorithm in Recommenders library into AzureML Designer. 

[AzureML Designer](https://docs.microsoft.com/en-us/azure/machine-learning/concept-designer) lets you visually connect datasets and modules on an interactive canvas to create machine learning models. 

![img](https://recodatasets.blob.core.windows.net/images/designer-drag-and-drop.gif)

One of the features of AzureML Designer is that it is possible for developers to integrate any python library to make it available as a module. In this notebook are are going to show how to integrate [SAR](sar_movielens.ipynb) and several other modules in Designer


## Installation

The first step is to install AzureML Designer SDK. Assuming that you have installed the Recommenders environment `reco_base` as explained in the [SETUP.md](../../SETUP.md), you need to install:
```bash
conda activate reco_base
pip install keyring artifacts-keyring
pip install azureml-designer-tools==0.1.10.post8799295 --extra-index-url=https://msdata.pkgs.visualstudio.com/_packaging/azureml-modules%40Local/pypi/simple/ 
```

## Module implementation

The scenario that we are going to reproduce in Designer, as a reference example, is the content of the [SAR quickstart notebook](sar_movielens.ipynb). In it, we load a dataset, split it into train and test sets, train SAR algorithm, predict using the test set and compute several ranking metrics (precision at k, recall at k, MAP and nDCG).

For the pipeline that we want to create in Designer, we need to build the following modules:

- Stratified splitter
- SAR training
- SAR prediction
- Precision at k
- Recall at k
- MAP
- nDCG

The python code is defined with a python entry and a yaml file. All the python entries and yaml files for this pipeline can be found in [reco_utils/azureml/azureml_designer_modules](../../reco_utils/azureml/azureml_designer_modules).


### Define python entry

To illustrate how a python entry is defined we are going to explain the [precision at k entry](../../reco_utils/azureml/azureml_designer_modules/precision_at_k_entry.py). A simplified version of the code is shown next:

```python
# Dependencies
from azureml.studio.core.data_frame_schema import DataFrameSchema
from azureml.studio.core.io.data_frame_directory import (
    load_data_frame_from_directory,
    save_data_frame_to_directory,
)
from reco_utils.evaluation.python_evaluation import precision_at_k

# First, the input variables of precision_at_k are defined as argparse arguments
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--rating-true", help="True DataFrame.")
    parser.add_argument("--rating-pred", help="Predicted DataFrame.")
    parser.add_argument(
        "--col-user", type=str, help="A string parameter with column name for user."
    )
    # ... more arguments
    args, _ = parser.parse_known_args()

    # This module has two main inputs from the canvas, the true and predicted labels
    # they are loaded into the runtime as a pandas DataFrame
    rating_true = load_data_frame_from_directory(args.rating_true).data
    rating_pred = load_data_frame_from_directory(args.rating_pred).data

    # The python function is instantiated and the computation is performed
    eval_precision = precision_at_k(rating_true, rating_pred)
    
    # To output the result to Designer
```


### Define yaml entry


